# Value-at-Risk for Stocks: Historical Simulation

### Lecture Notes by Jakov Ivan S. Dumbrique (jdumbrique@ateneo.edu)

MATH 100.2: Topics in Financial Mathematics II \
First Semester, S.Y. 2021-2022 \
Ateneo de Manila University

In [1]:
import numpy as np # Numerical Computing
import pandas as pd # Data wrangling

## Introduction to Historical Simulation

1. The primary basis of the Delta-Normal approach in the computation of the VaR is the assumption that asset price returns (or portfolio returns) are normally distributed.
2. Historical data can also be used to estimate VaR for stock portfolios without assuming a particular parametric distribution for the asset return series. This particular procedure is known as **historical simulation**.

Consider a financial institution with a position of $N$ shares on a stock $E$ with price $S_{0}$ today. Let $S_1$ be the price per share of $E_1$ tomorrow.

Let $R_1 = \ln(S_1/S_0)$ be the continuously compounded return on the asset observed tomorrow.
For sufficiently small $R_1$, we assume that $$\Delta P = P_1-P_0 = N(S_1-S_0) \approx N S_0 R_1.$$

In historical simulation, it is also assumed that $R_1$ is a random variable whose values consist of the one-day asset returns observed historically. However, unlike in Delta-Normal Approach, we don't assume $R_1$ to be normally distributed.

###  Step 1: Generate historical one-day returns $\{R_{1,j}\}$
Consider the time series $\{S_{-t}\}$, $t=0,1,2,\dots,M$, where $S_{-t}$ indicates the stock price $t$ trading days ago and $M+1$ is the length of the time series.

Define
\begin{equation}
R_{1,j} = \ln\left(\frac{S_{-j}}{S_{-(j+1)}}\right), j=0,1,\dots,M-1
\end{equation}
and so the series $\{R_{1,j}\}$ contains all possible values of the random variable $R_1$, with $R_{1,j}$ representing the $j$th scenario for $R_1$.

<h3 style='color:green'> Example on Single-Asset Portfolio</h3> 
Today is February 23, 2018. You are a portfolio risk manager who is assigned to analyze the market risk of a portfolio of 700 PLDT (TEL) shares. Determine the portfolio's one-day 99% VaR using historical simulation approach.
 


In [2]:
TEL_df = pd.read_csv("https://raw.githubusercontent.com/ateneomathdept/math100.2_2021Sem1/main/data/lectures/TEL_2018.csv")
TEL_df

,dt,close
0,2/23/18,1488.74
1,2/22/18,1510.86
2,2/21/18,1513.72
3,2/20/18,1536.65
4,2/16/18,1476.37
...,...,...
243,3/2/17,1430.29
244,3/1/17,1408.94
245,2/28/17,1394.35
246,2/27/17,1374.93


In [3]:
def get_return(df, d):
    """
    df is the original df
    
    appends returns series to df
    """
    df["previous"] = df["close"].shift(-d)
    df["return"] = np.log(df["close"]/df["previous"])
    return df 

In [4]:
TEL_df = get_return(TEL_df, d=1)
TEL_df

,dt,close,previous,return
0,2/23/18,1488.74,1510.86,-0.014749
1,2/22/18,1510.86,1513.72,-0.001891
2,2/21/18,1513.72,1536.65,-0.015035
3,2/20/18,1536.65,1476.37,0.040018
4,2/16/18,1476.37,1475.71,0.000447
...,...,...,...,...
243,3/2/17,1430.29,1408.94,0.015040
244,3/1/17,1408.94,1394.35,0.010409
245,2/28/17,1394.35,1374.93,0.014026
246,2/27/17,1374.93,1367.68,0.005287


### Step 2: Get historical changes in portfolio values $\{\Delta P_j\}$
In scenario $j$, the stock price tomorrow is approximated as
\begin{equation}
S_{1,j} = S_0 e^{R_{1,j}}.
\end{equation}

Consequently, the change in portfolio value under scenario $j$ is given by the approximation
\begin{equation}
\Delta P_j = N (S_{1,j}-S_0) \approx NS_0 R_{1,j}.
\end{equation}

Like the random variable $R_1$, $\Delta P$ can be represented as a random variable with $M$ possible scenarios denoted by $\{\Delta P_j\}$.


In [5]:
def get_change_in_value_df(df, N):
    """
    df is the output of get_returns_df 
    
    appends change in portfolio value series to df
    """
    S0 = df.loc[0, "close"]
    df["change_in_value"] = N * S0 * df["return"]
    return df

In [6]:
TEL_df = get_change_in_value_df(TEL_df, N=700)
TEL_df

,dt,close,previous,return,change_in_value
0,2/23/18,1488.74,1510.86,-0.014749,-15370.094698
1,2/22/18,1510.86,1513.72,-0.001891,-1970.824622
2,2/21/18,1513.72,1536.65,-0.015035,-15667.749059
3,2/20/18,1536.65,1476.37,0.040018,41703.842890
4,2/16/18,1476.37,1475.71,0.000447,465.975103
...,...,...,...,...,...
243,3/2/17,1430.29,1408.94,0.015040,15673.009003
244,3/1/17,1408.94,1394.35,0.010409,10847.710303
245,2/28/17,1394.35,1374.93,0.014026,14616.263663
246,2/27/17,1374.93,1367.68,0.005287,5509.622658


### Step 3: Get the (1-alpha)th percentile of the ordered array of historical changes in portfolio values
 
The one-day 99\% VaR of the portfolio thus corresponds to the scenario $j^*$, given by $\Delta P_{j^*}$, in the 1st percentile of the scenario space $\{\Delta P_j\}$. 

$\Delta P_{j^*}$ represents the worst possible movement in portfolio value with probability 99\%. 

If the $\Delta P_j$s are arranged in ascending order, the one-day 99\% VaR for the portfolio is the $\lfloor 0.01M \rfloor$th smallest value in the array.


In [7]:
M = len(TEL_df)-1
M

247

In [8]:
0.01*M

2.47

In [9]:
k = int(np.floor(0.01*M))
k

2

In [10]:
TEL_df["change_in_value"]

0     -15370.094698
1      -1970.824622
2     -15667.749059
3      41703.842890
4        465.975103
           ...     
243    15673.009003
244    10847.710303
245    14616.263663
246     5509.622658
247             NaN
Name: change_in_value, Length: 248, dtype: float64

In [11]:
var = abs(TEL_df["change_in_value"].sort_values(ignore_index=True)[k-1])
var

60730.66451735293

In [12]:
def get_kth_percentile_discrete(df, d, alpha):
    """
    this returns the (1-alpha)th percentile of the ordered array of historical changes in portfolio values
    """
    M = len(df)-d
    k = int(np.floor((1-alpha)*M))
    var = abs(df["change_in_value"].sort_values(ignore_index=True)[k-1])
    
    return var

In [13]:
get_kth_percentile_discrete(TEL_df, d=1, alpha=0.99)

60730.66451735293

In [14]:
def d_day_alpha_percent_VaR_single_stock_historical(
    df, N, d, alpha
):
    """Returns the d-day 100(alpha)% VaR of a single stock using Historical Simulation Approach.
    
    Parameters
    ----------
    df : pandas.DataFrame
        has two columns: (1) dt [str] and (2) closing price [float]
        assumes the dates are arranged from newest to oldest, and the date today is the date on the first row  
    N : int
        number of shares for the sole stock
    d : int
        the value to be used in calculating the d-day VaR (e.g. 1-day, 5-day)
    alpha : float
        the value to be used in calculting the 100(alpha)% VaR (e.g. 0.99, 0.95)
    
    Returns
    -------
    float (2 decimal places)
        d-day 100(alpha)% VaR of a single stock using Historical Simulation Approach
    """
    
    # step 1: generate your historical returns
    df = get_return(df, d)
    # Step 2: get your historical changes in portfolio values
    df = get_change_in_value_df(df, N)
    # Step 3: get the (1-p/100)th percentile of the ordered array of historical changes in portfolio values
    var = get_kth_percentile_discrete(df, d, alpha)
    
    return round(var, 2)

In [15]:
TEL_df = pd.read_csv("https://raw.githubusercontent.com/ateneomathdept/math100.2_2021Sem1/main/data/lectures/TEL_2018.csv")
TEL_df

,dt,close
0,2/23/18,1488.74
1,2/22/18,1510.86
2,2/21/18,1513.72
3,2/20/18,1536.65
4,2/16/18,1476.37
...,...,...
243,3/2/17,1430.29
244,3/1/17,1408.94
245,2/28/17,1394.35
246,2/27/17,1374.93


In [16]:
d_day_alpha_percent_VaR_single_stock_historical(df=TEL_df, N=700, d=1, alpha=0.99)

60730.66

<h3 style='color:green'> Exercise on N-Asset Portfolio</h3> 
Today is February 23, 2018. You are a portfolio risk manager who is assigned to analyze the market risk of a portfolio of 700 PLDT (TEL) shares, 300 Ayala Corporation (AC) shares, and 500 URC shares. What is the portfolio's one-day 99% VaR using historical approach?